In [1]:

#1. 음악 데이터 파일 로드에 필요한 라이브러리 
import IPython.display
import pandas as pd
import numpy as np

# 음악 분석 라이브러리
import librosa
import librosa.display
import matplotlib.pyplot as plt

In [2]:
#2. 음악 정보 파일 불러오기
# music_inform.csv 

music_inform=pd.read_csv('music_inform.csv')

display(music_inform)

,Music,Genre,ID,Format,Path
0,Contact High - RKVC,RnBSoul_Calm,R_C1,mp3,RnBSoul_Calm/R_C1.mp3
1,QuangerineCream - Noir Et Blanc Vie,RnBSoul_Calm,R_C2,mp3,RnBSoul_Calm/R_C2.mp3
2,TrueArtRealAffectionPart4 - Noir Et Blanc Vie,RnBSoul_Calm,R_C3,mp3,RnBSoul_Calm/R_C3.mp3
3,Wallflowers - Bad Snacks,RnBSoul_Calm,R_C4,mp3,RnBSoul_Calm/R_C4.mp3
4,Where To_ - Birocratic,RnBSoul_Calm,R_C5,mp3,RnBSoul_Calm/R_C5.mp3
5,Goji Berry Beat - Nana Kwabena,Hiphop_Funky,H_F1,mp3,Hiphop_Funky/H_F1.mp3
6,Madam Wahala Beat - Nana Kwabena,Hiphop_Funky,H_F2,mp3,Hiphop_Funky/H_F2.mp3
7,Pretty Boy - DJ Freedem,Hiphop_Funky,H_F3,mp3,Hiphop_Funky/H_F3.mp3
8,Road Tripzzz - Ofshane,Hiphop_Funky,H_F4,mp3,Hiphop_Funky/H_F4.mp3
9,Twisted Bandits All Around Me - NoMBe,Hiphop_Funky,H_F5,mp3,Hiphop_Funky/H_F5.mp3


In [3]:
#3. 분석에 필요한 열만 추출 : music_data
#ID, Path

music_data=music_inform[['ID','Path']]

display(music_data)

,ID,Path
0,R_C1,RnBSoul_Calm/R_C1.mp3
1,R_C2,RnBSoul_Calm/R_C2.mp3
2,R_C3,RnBSoul_Calm/R_C3.mp3
3,R_C4,RnBSoul_Calm/R_C4.mp3
4,R_C5,RnBSoul_Calm/R_C5.mp3
5,H_F1,Hiphop_Funky/H_F1.mp3
6,H_F2,Hiphop_Funky/H_F2.mp3
7,H_F3,Hiphop_Funky/H_F3.mp3
8,H_F4,Hiphop_Funky/H_F4.mp3
9,H_F5,Hiphop_Funky/H_F5.mp3


In [4]:
#4. 모든 곡들의 spectral_centroid저장 
n=len(music_data) #데이터 수
spectral_centroid=[]


for i in range(n):
    music_name=music_data.loc[i]
    music_path=music_name[1]
    
    y,sr=librosa.load(music_path,sr=44100,duration=10)
    IPython.display.Audio(data=y,rate=sr)


    #spectral_centroid 추출
    s_c=librosa.feature.spectral_centroid(y,sr=sr)
    spectral_centroid.append(s_c)
    
frame_num=len(spectral_centroid[0][0]) #프레임 수

c:\users\yeongeun\appdata\local\programs\python\python38-32\lib\site-packages\librosa\core\audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


In [5]:
#5. DTW함수
def DTW(a,b,length):
    #5-1. dtw를 구하기 위한 행렬
    dtw=[[0 for i in range(length)] for j in range(length)]
    
    #5-2. dtw 첫행값 구하기
    dtw[0][0]=abs(a[0]-b[0])
    
    for dj in range(1,length):
        dtw[0][dj]=dtw[0][dj-1]+abs(a[0]-b[dj])
    
    #5-3. dtw 첫열값 구하기
    for di in range(1,length):
        dtw[di][0]=dtw[di-1][0]+abs(a[di]-b[0])
        
    #5-4. dtw 채우기
    for di in range(1,length):
        for dj in range(1,length):
            min_dtw=min(min(dtw[di-1][dj-1],dtw[di-1][dj]),dtw[di][dj-1])
            
            dtw[di][dj]=min_dtw+abs(a[di]-b[dj])
            
    #5-5. 최종 dtw 유사거리
    return dtw[length-1][length-1]
    
    
    

In [6]:
#6. spectral_centroid 유사 구해보기
s_c_similarity=[[0 for i in range(n)] for j in range(n)]

for i in range(n):
    for j in range(i+1,n):
        s_c_similarity[i][j]=DTW(spectral_centroid[i][0],spectral_centroid[j][0],frame_num)
        s_c_similarity[j][i]=s_c_similarity[i][j]
        
        #현재 진행상황 파악하기 위한 print
        print('[',i,']','[',j,']의 유사도 : ', s_c_similarity[i][j])


[ 0 ] [ 1 ]의 유사도 :  406092.04181047203
[ 0 ] [ 2 ]의 유사도 :  613418.3358934317
[ 0 ] [ 3 ]의 유사도 :  741606.6372611936
[ 0 ] [ 4 ]의 유사도 :  617620.612628722
[ 0 ] [ 5 ]의 유사도 :  686890.7735057957
[ 0 ] [ 6 ]의 유사도 :  506328.2802595861
[ 0 ] [ 7 ]의 유사도 :  372048.45580311975
[ 0 ] [ 8 ]의 유사도 :  1211566.827457891
[ 0 ] [ 9 ]의 유사도 :  786414.1374242918
[ 0 ] [ 10 ]의 유사도 :  847186.8141677576
[ 0 ] [ 11 ]의 유사도 :  625231.8938389891
[ 0 ] [ 12 ]의 유사도 :  1026440.5613887468
[ 0 ] [ 13 ]의 유사도 :  728406.8855097136
[ 0 ] [ 14 ]의 유사도 :  791986.4034383441
[ 1 ] [ 2 ]의 유사도 :  581977.7039344746
[ 1 ] [ 3 ]의 유사도 :  480735.2917386601
[ 1 ] [ 4 ]의 유사도 :  463258.03016515047
[ 1 ] [ 5 ]의 유사도 :  369057.3966419802
[ 1 ] [ 6 ]의 유사도 :  467239.277344505
[ 1 ] [ 7 ]의 유사도 :  382197.0711588126
[ 1 ] [ 8 ]의 유사도 :  1489033.1380421557
[ 1 ] [ 9 ]의 유사도 :  467452.598206343
[ 1 ] [ 10 ]의 유사도 :  789825.271527829
[ 1 ] [ 11 ]의 유사도 :  515984.60693886824
[ 1 ] [ 12 ]의 유사도 :  996099.2436109356
[ 1 ] [ 13 ]의 유사도 :  561944.3977114017
[

In [7]:
#7. 3곡정도씩 유사곡 리스트 만들어보기 
#r_num : 유사곡 수 : 5
#music_num : 자기 자신 번호
#s: s_c_similarity[music_num]

def recommendation(music_num,s,r_num):
    s_list=[]
    
    for i in range(len(s)):
        if music_num==i:
            continue
        s_list.append([s[i],i])
        
    s_list=sorted(s_list)
    
    return [s_list[i][1] for i in range(r_num)]


r_num=5
for i in range(n):
    print(i,'곡의 spectral_centroid 유사 곡 : ',recommendation(i,s_c_similarity[i],r_num))
    
    

0 곡의 spectral_centroid 유사 곡 :  [7, 1, 6, 2, 4]
1 곡의 spectral_centroid 유사 곡 :  [5, 7, 0, 4, 6]
2 곡의 spectral_centroid 유사 곡 :  [9, 10, 3, 5, 6]
3 곡의 spectral_centroid 유사 곡 :  [9, 2, 5, 6, 14]
4 곡의 spectral_centroid 유사 곡 :  [5, 11, 1, 13, 7]
5 곡의 spectral_centroid 유사 곡 :  [9, 3, 14, 1, 2]
6 곡의 spectral_centroid 유사 곡 :  [3, 9, 7, 14, 2]
7 곡의 spectral_centroid 유사 곡 :  [0, 1, 6, 14, 5]
8 곡의 spectral_centroid 유사 곡 :  [0, 4, 12, 11, 13]
9 곡의 spectral_centroid 유사 곡 :  [2, 3, 10, 5, 6]
10 곡의 spectral_centroid 유사 곡 :  [2, 9, 3, 5, 14]
11 곡의 spectral_centroid 유사 곡 :  [4, 1, 0, 5, 7]
12 곡의 spectral_centroid 유사 곡 :  [13, 7, 1, 0, 4]
13 곡의 spectral_centroid 유사 곡 :  [4, 1, 7, 5, 14]
14 곡의 spectral_centroid 유사 곡 :  [3, 5, 9, 6, 2]


In [9]:
#7. 곡별 추천 리스트 파일로 저장하기

#recomsize: 추천 리스트
#header 생성
recomsize=['recom'+str(i) for i in range(1,r_num+1)]
header=['music_id']+recomsize

recom_list=[]

for i in range(n):
    recom_list.append([i]+recommendation(i,s_c_similarity[i],r_num))
    
recom_df=pd.DataFrame(recom_list,columns=header)
display(recom_df)

#csv파일로 저장
recom_df.to_csv('spectral_centroid_recommendation.csv',header=True,index=False)

,music_id,recom1,recom2,recom3,recom4,recom5
0,0,7,1,6,2,4
1,1,5,7,0,4,6
2,2,9,10,3,5,6
3,3,9,2,5,6,14
4,4,5,11,1,13,7
5,5,9,3,14,1,2
6,6,3,9,7,14,2
7,7,0,1,6,14,5
8,8,0,4,12,11,13
9,9,2,3,10,5,6
